In [1]:
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession
    .builder
    .config("spark.driver.memory", "10g")
    .master("local[*]")
    .appName("6_1_dynamic_partition_pruning")
    .getOrCreate()
)
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/19 18:04:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df_listening_actv_pt = spark.read.parquet("/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data/partitioning/partitioned/listening_activity_pt")
df_listening_actv_pt.show(5, False)

+-----------+-------+--------------------------+---------------+-----------+-----------+
|activity_id|song_id|Listen_time               |listen_duration|Listen_hour|Listen_date|
+-----------+-------+--------------------------+---------------+-----------+-----------+
|4456       |16     |2023-07-18 10:15:47.023264|151            |10         |2023-07-18 |
|4457       |65     |2023-07-18 10:15:47.023264|181            |10         |2023-07-18 |
|4458       |60     |2023-07-18 10:15:47.023264|280            |10         |2023-07-18 |
|4459       |3      |2023-07-18 10:15:47.023264|249            |10         |2023-07-18 |
|4460       |45     |2023-07-18 10:15:47.023264|130            |10         |2023-07-18 |
+-----------+-------+--------------------------+---------------+-----------+-----------+
only showing top 5 rows


In [8]:
# df_listening_actv_pt.explain()÷

In [9]:
df_songs = spark.read.csv("/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data/partitioning/raw/Spotify_Songs.csv", header=True, inferSchema=True)
df_songs.printSchema()

root
 |-- song_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: integer (nullable = true)
 |-- release_date: timestamp (nullable = true)



In [10]:
df_songs = (
    df_songs
    .withColumnRenamed("release_date", "release_datetime")
    .withColumn("release_date", F.to_date("release_datetime", "yyyy-MM-dd HH:mm:ss.SSSSSS"))
)
df_songs.show(5, False)
df_songs.printSchema()

+-------+------+---------+--------------------------+------------+
|song_id|title |artist_id|release_datetime          |release_date|
+-------+------+---------+--------------------------+------------+
|1      |Song_1|2        |2021-10-15 10:15:47.006571|2021-10-15  |
|2      |Song_2|45       |2020-12-07 10:15:47.006588|2020-12-07  |
|3      |Song_3|25       |2022-07-11 10:15:47.006591|2022-07-11  |
|4      |Song_4|25       |2019-03-09 10:15:47.006593|2019-03-09  |
|5      |Song_5|26       |2019-09-07 10:15:47.006596|2019-09-07  |
+-------+------+---------+--------------------------+------------+
only showing top 5 rows
root
 |-- song_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: integer (nullable = true)
 |-- release_datetime: timestamp (nullable = true)
 |-- release_date: date (nullable = true)



In [12]:
# Pick songs released in 2020
df_selected_songs = df_songs.filter(F.col("release_date") > F.lit("2019-12-31"))


df_listening_actv_of_selected_songs = df_listening_actv_pt.join(
    df_selected_songs, 
    on=(df_songs.release_date == df_listening_actv_pt.Listen_date) & (df_songs.song_id == df_listening_actv_pt.song_id), 
    how="inner"
)

In [ ]:
df_listening_actv_of_selected_songs.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [Listen_date#83, song_id#79], [release_date#127, song_id#121], Inner, BuildRight, false
   :- Filter isnotnull(song_id#79)
   :  +- FileScan parquet [activity_id#78,song_id#79,Listen_time#80,listen_duration#81,Listen_hour#82,Listen_date#83] Batched: true, DataFilters: [isnotnull(song_id#79)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data..., PartitionFilters: [isnotnull(Listen_date#83), dynamicpruningexpression(Listen_date#83 IN dynamicpruning#148)], PushedFilters: [IsNotNull(song_id)], ReadSchema: struct<activity_id:int,song_id:int,Listen_time:timestamp,listen_duration:int,Listen_hour:int>
   :        +- SubqueryAdaptiveBroadcast dynamicpruning#148, [0], true, Project [song_id#121, title#122, artist_id#123, release_date#124 AS release_datetime#126, cast(gettimestamp(release_date#124, yyyy-MM-dd HH:mm:ss.SSSSSS, TimestampType, try_t

25/08/20 09:15:02 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:131)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:700)
	at org.apache.spark.storage.BlockManagerMasterE